In [ ]:
import os
import pandas as pd

In [15]:
# Определяем путь в которой работаем
main_path = os.getcwd()

# Загружаем необходимые для работы файлы
logs = pd.read_csv(main_path+'\\logs.csv')
genres = pd.read_csv(main_path+'\\genres.csv')
movies = pd.read_csv(main_path+'\\movies.csv')
staff = pd.read_csv(main_path+'\\staff.csv')
countries = pd.read_csv(main_path+'\\countries.csv')

# str -> list
movies['genres'] = movies['genres'].apply(eval)
movies['countries'] = movies['countries'].apply(eval)
movies['staff'] = movies['staff'].apply(eval)


In [14]:
!pip install pymorphy2 nltk

import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer

nltk.download('punkt')
from pymorphy2 import MorphAnalyzer
from collections import defaultdict
from itertools import chain
from sklearn.preprocessing import MinMaxScaler


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/55.5 kB ? eta -:--:--
     ---------------------- ----------------- 30.7/55.5 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 55.5/55.5 kB 580.0 kB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.5 MB 2.6 MB/s eta 0:00:01
     ---- ----------------------------------- 0.2/1.5 MB 2.0 MB/s eta 0:00:01
     ----- ---------------------------------- 0.2/1.5 MB 2.3 MB/s eta 0:00:01
     ------------ --------------------------- 0.5/1.5 MB 2.7 MB/s eta 0:00:01
     --------------- ------------------------ 0.6/1.5 MB 2.7 MB/s eta 0:00:01
     ----------------------- ---------------- 0.9/1.5 MB 3.5 MB/s eta 0:00:01
     ----------------------- ---------------- 0.9/1.5 MB 3.5 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Пользователь\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [16]:
def MyLemmatizer(texts:tuple):
  morph = MorphAnalyzer()

  def prepare(text: str):
      prep_text = text.lower()
      prep_text = re.sub(r'\b\w\b', '', prep_text)
      prep_text = re.sub(r'\b\w\w\b', '', prep_text)
      prep_text = re.sub(r'[^\w\s]', ' ', prep_text)
      prep_text = (i for i in prep_text.split() if i != '')
      prep_text = (i for i in prep_text if i not in stopwords)
      return tuple(prep_text)

  old_word = prepare(' '.join(texts))

  norm_word = (
      max(((r.score, r.normal_form) for r in morph.parse(word)), key=lambda x: x[0])[1]
      for word in old_word
  )

  norm_dict = dict(zip(old_word, norm_word))
  word_dict = dict(Counter(old_word))
  max_count = max(word_dict.values())
  word_dict = {word: count / max_count for word, count in word_dict.items()}

  result = [
      (np.mean([word_dict.get(norm_dict.get(word, None), 0.0) for word in prepare(text)]))
      if prepare(text)
      else 0.0
      for text in texts
  ]

  return result

In [ ]:
class MyLemmatizer:
  def __init__(self):
    self.norm_dict = {}
    self.word_dict = {}
    self.morph = MorphAnalyzer()

  def prepare(self, text:str):
    prep_text = text.lower()
    prep_text = re.sub(r'\b\w\w\b', '', prep_text)
    prep_text = re.sub(r'\b\w\b', '', prep_text)
    prep_text = re.sub(r'[^\w\s]', ' ', prep_text)
    prep_text = [i for i in prep_text.split(' ') if i!='']
    prep_text = [i for i in prep_text if i not in stopwords]
    return prep_text

  def fit(self, texts:list):
    word_list = []
    for word in self.prepare(' '.join(texts)):
      m_score, m_words = zip(*((r.score, r.normal_form) for r in self.morph.parse(word)))
      max_score_index = max(range(len(m_score)), key=m_score.__getitem__)
      norm_word = m_words[max_score_index]
      word_list.append(norm_word)
      self.norm_dict[word] = norm_word

    c = Counter(word_list)
    self.word_dict = {k:v/max(c.values()) for k, v in c.items()}

  def transform(self, text:str):
    lem_text = []
    if text=='':
      return 0.0
    else:
      prepare_text = self.prepare(text)
      if len(prepare_text)==0:
        return 0.0
      else:
        for word in prepare_text:
          if word in self.norm_dict.keys() and self.norm_dict[word] in self.word_dict.keys():
            lem_text.append(self.word_dict[self.norm_dict[word]])
          else:
              lem_text.append(0.0)

      return np.mean(lem_text)

In [ ]:
movies.description = movies.description.fillna('')
description = movies.loc[train_id, 'description'].tolist()
lematizer = MyLemmatizer()
lematizer.fit(description)
movies.description = [lematizer.transform(text) for text in tqdm(movies.description, desc = '     DESCRIPTION')]

In [ ]:
class attr_count:
  def __init__(self):
    self.items_dict = {}

  def fit(self, items:list):
    items = list(chain.from_iterable(items))
    c = Counter(items)
    self.items_dict = {k:v/max(c.values()) for k, v in c.items()}

  def transform(self, items:list):
    if len(items) == 0:
      return 0.0
    else:
      return np.mean([self.items_dict[item] if item in self.items_dict.keys() else 0.0 for item in items])


def data_prepare(user_id:int):
  global log
  scaler = MinMaxScaler()
  movies_list = list(set(log.loc[log.user_id==user_id, 'movie_id'].tolist()))
  train_id, test_id = train_test_split(movies_list, test_size=0.2)

  print('        LOAD')
  movies = pd.read_csv('/content/movies.csv')
  # output.clear(wait=True)
  # print(print_text:= '✓ LOAD')

  print('       DATA PREPARE:\n')
  movies.year = pd.to_datetime(movies.year, format='%Y-%m-%d')
  movies.year = [date.timestamp() for date in tqdm(movies.year, desc = '            YEAR')]

  movies.description = movies.description.fillna('')
  description = movies.loc[train_id, 'description'].tolist()
  lematizer = MyLemmatizer()
  lematizer.fit(description)
  movies.description = [lematizer.transform(text) for text in tqdm(movies.description, desc = '     DESCRIPTION')]

  movies['genres'] = movies['genres'].apply(eval)
  ac = attr_count()
  ac.fit(movies.loc[train_id, 'genres'].tolist())
  movies.genres = [ac.transform(g) for g in tqdm(movies.genres, desc = '          GENRES')]

  movies['countries'] = movies['countries'].apply(eval)
  ac = attr_count()
  ac.fit(movies.loc[train_id, 'countries'].tolist())
  movies['countries'] = [ac.transform(c) for c in tqdm(movies['countries'], desc = '       COUNTRIES')]

  movies['staff'] = movies['staff'].apply(eval)
  ac = attr_count()
  ac.fit(movies.loc[train_id, 'staff'].tolist())
  movies['staff'] = [ac.transform(s) for s in tqdm(movies['staff'], desc = '           STAFF')]

  movies = movies.drop(['id', 'name', 'date_publication', 'year'], axis=1)
  # output.clear(wait=True)
  # print(print_text:= print_text+'\n✓ DATA PREPARE')

  print('       TRANSFORM TO TENSOR')
  x_train = torch.tensor(movies.values, dtype=torch.float32)
  user_log:pd.DataFrame = log.loc[log.user_id == user_id]
  y_train = [[sum(user_log.loc[user_log.movie_id == m_id, 'duration'])] if m_id in train_id else [0.0] for m_id in movies.index.tolist()]
  y_train = scaler.fit_transform(y_train)
  y_train = torch.tensor(y_train, dtype=torch.float32, requires_grad=True)

  x_test = torch.tensor(movies.values, dtype=torch.float32)
  y_test = test_id
  # output.clear(wait=True)
  # print(print_text:= print_text+'\n✓ DATA TRANSFORM TO TENSOR')

  return (x_train, y_train), (x_test, y_test)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import re
from tqdm import tqdm
import random
import itertools
import time
from matplotlib import pyplot as plt
import seaborn as sns
from google.colab import output
from decimal import Decimal

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize, Normalizer

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

from collections import Counter

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
stopwords = tuple(nltk_stopwords.words('russian'))

!pip install torch torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

